## Weavechain hashes storage in ICP

In this demo notebook we will showcase storing integrity proofs on ICP:

- we will create a table in a data collection
- mark some fields as personal information that is not to be shared
- write a record
- read it locally (and be able to see all fields)
- read hashes stored on ICP
   
A Weavechain node is installed and preconfigured to support this scenario

### 1. Create an API session

In [47]:
import pandas as pd

from weaveapi.records import *
from weaveapi.options import *
from weaveapi.filter import *
from weaveapi.weaveh import *

WEAVE_CONFIG = "config/demo_client_local_rolodex.config"
nodeApi, session = connect_weave_api(WEAVE_CONFIG)

scope = "rolodex"
table = "directory"

{"res":"ok","data":"pong 1697632804575"}


### 2. Create a local table

- we drop the existing table if already existing and re-create it from scratch
- a weavechain node can also connect to existing tables, reading their structure, but in this case we create it via the API

In [48]:
layout = { 
    "columns": { 
        "id": { "type": "LONG", "isIndexed": True, "isUnique": True, "isNullable": False },
        "name_nickname": { "type": "STRING" },
        "name_last": { "type": "STRING" },
        "name_first": { "type": "STRING" },
        "birthday": { "type": "STRING" },
        "email_personal": { "type": "STRING" },
        "phone_number": { "type": "STRING" },
        "address_country": { "type": "STRING" },
        "address_summary": { "type": "STRING" },
        "address_timezone": { "type": "STRING" },
        "linkedin_url": { "type": "STRING" },
        "discord_username": { "type": "STRING" },
        "telegram_username": { "type": "STRING" },
        "ethereum_wallet_address": { "type": "STRING" }
    }, 
    "idColumnIndex": 0, 
    "isLocal": False,
    "applyReadTransformations": True
}

nodeApi.dropTable(session, scope, table).get()
reply = nodeApi.createTable(session, scope, table, CreateOptions(False, False, layout)).get()
print(reply)

{'res': 'ok', 'target': {'operationType': 'CREATE_TABLE', 'organization': 'icprolodex', 'account': 'icprolodex', 'scope': 'rolodex', 'table': 'directory'}}


### 3. Mark some fields for erasure

- the purpose is to protect certain fields when shared

In [49]:
reply = nodeApi.getTableDefinition(session, scope, table).get()
#print(reply)
layout = json.loads(reply["data"])["layout"]

newLayout = layout.copy()
del newLayout["layout"]
del newLayout["indexes"]
del newLayout["columnNames"]
newLayout["columns"] = { i["columnName"]: i for i in layout["columns"]}

newLayout["columns"]["phone_number"]["readTransform"] = "ERASURE"
newLayout["columns"]["address_summary"]["readTransform"] = "ERASURE"
newLayout["columns"]["ethereum_wallet_address"]["readTransform"] = "ERASURE"
newLayout["columns"]["birthday"]["readTransform"] = "ERASURE"

reply = nodeApi.updateLayout(session, scope, table, newLayout).get()
print(reply)

{'res': 'ok', 'target': {'operationType': 'UPDATE_LAYOUT', 'organization': 'icprolodex', 'account': 'icprolodex', 'scope': 'rolodex', 'table': 'directory'}}


### 4. Write few records in the local storage

In [50]:
records = Records(table, [ 
    [ None, 'Nickname', 'Last Name', 'First name', '1980-01-01', 'email@gmail.com', '+40712345678', 'US', 'Secret', 'EST', 'https://www.linkedin.com/in/linkedin/', 'discord#1234', '@telegram', '0xwallet' ],
    [ None, 'jdoe', 'John', 'Doe', '1990-05-01', 'johndoe@gmail.com', '+44712345678', 'US', 'Secret', 'EST', 'https://www.linkedin.com/in/linkedin/', 'discord#jdoe', '@jdoe', '0xwallet' ],
    [ None, 'jndoe', 'Jane', 'Doe', '2001-03-01', 'janedoe@gmail.com', '+112345678', 'US', 'Secret', 'EST', 'https://www.linkedin.com/in/linkedin/', 'discord#1234', '@telegram', '0xwallet' ],
    [ None, 'foobar', 'Foo', 'Bar', '1989-12-01', 'fbar@gmail.com', '+122345678', 'US', 'Secret', 'EST', 'https://www.linkedin.com/in/linkedin/', 'discord#1234', '@telegram', '0xwallet' ]
])
reply = nodeApi.write(session, scope, records, WRITE_DEFAULT).get()
print(reply)

{'res': 'ok', 'target': {'operationType': 'WRITE', 'organization': 'icprolodex', 'account': 'icprolodex', 'scope': 'rolodex', 'table': 'directory'}, 'data': 'weaved8xLnTMp5B5GtJwDQvc1u7K4fwPc2ry7iDieyCdJRHcG,4ABlbIDxrix6H4qT+B6SPNeUs4oCCeW+O0KRd27TfY8=,3Uk9S2QVLWyYNmx1ofhU7KiQsiyQpRkffyUMFoLc6KZp15tVicgdbEWSHi31YsWaX47UMiZKczkTiMRxtGczQJwU', 'ids': '1 2 3 4'}


### 5. Read the local record, from the local storage

- since we read with the owner key and from the local node, we expect the records to have all fields visible

In [51]:
scope = "rolodex"
table = "directory"

reply = nodeApi.read(session, scope, table, None, READ_DEFAULT_NO_CHAIN).get()
#print(reply)
df = pd.DataFrame(reply["data"])

df.head()

,id,name_nickname,name_last,name_first,birthday,email_personal,phone_number,address_country,address_summary,address_timezone,linkedin_url,discord_username,telegram_username,ethereum_wallet_address
0,1,Nickname,Last Name,First name,1980-01-01,email@gmail.com,+40712345678,US,Secret,EST,https://www.linkedin.com/in/linkedin/,discord#1234,@telegram,0xwallet
1,2,jdoe,John,Doe,1990-05-01,johndoe@gmail.com,+44712345678,US,Secret,EST,https://www.linkedin.com/in/linkedin/,discord#jdoe,@jdoe,0xwallet
2,3,jndoe,Jane,Doe,2001-03-01,janedoe@gmail.com,+112345678,US,Secret,EST,https://www.linkedin.com/in/linkedin/,discord#1234,@telegram,0xwallet
3,4,foobar,Foo,Bar,1989-12-01,fbar@gmail.com,+122345678,US,Secret,EST,https://www.linkedin.com/in/linkedin/,discord#1234,@telegram,0xwallet


### 6. Read hashes

In [61]:
reply = nodeApi.hashes(session, scope, table, None, READ_DEFAULT_NO_CHAIN).get()
dh = pd.DataFrame([ [x, reply["data"][x]] for x in list(reply["data"]) ], columns=["ID", "Hash"])

display(dh)

,ID,Hash
0,1,Dj9xMufdcrM4uysMPi4oVmwQheSsd2CVI+j0tKp4ZXc=


### 7. Read merkle root for the table

In [54]:
from binarytree import Node

def showtree(tree):
    prev = None
    root = None
    lvl = 1
    for l in tree.split(";"):
        level = l.split(",")
        print(lvl, level)
        lvl += 1
        nodes = []
        for i in range(len(level)):
            pidx = int(i / 2)
            node = Node(level[i][:3] + ".." + level[i][-3:])
            if root is None:
                root = node
            nodes.append(node)
            if prev is not None:
                parent = prev[pidx]
                if parent.left is None:
                    parent.left = node
                else:
                    parent.right = node
        prev = nodes
    
    print(root)

salt = "salt1234" # Same salt used for records hashes, this can be improved to have different salts for *each distinct writer*
digest = "SHA-256"

filter = Filter(None, None, None, None, [ "name_nickname","name_last","name_first","birthday","email_personal","phone_number","address_country","address_summary","address_timezone","linkedin_url","discord_username","telegram_username","ethereum_wallet_address" ])
reply = nodeApi.merkleTree(session, scope, table, filter, salt, digest, READ_DEFAULT_NO_CHAIN).get()
tree = reply["data"]["tree"]
rootHash = reply["data"]["rootHash"]
ts = reply["data"]["timestamp"]
rootHashSignature = reply["data"]["signature"]

print("Generated at", ts)
print("Root Hash", rootHash)
print("Signature", rootHashSignature)
print("")
showtree(tree)
# We've built a Merkle Tree at a specific time, signed by the node that created it.
# The Merkle Leaves are salted hashes of the data in the table. Binance did an enumeration of coins.

Generated at 1697632831936
Root Hash EnwbMZFiuCEo2MJ9YEp3PDTqopWZqQ9VLi5x2zASZpX5
Signature 2YnBHRFeqaj6iRrVwVWSFE5QFogtZcgAvRHHn3y7i6ntPWkU9xh1CtgBMiRXSrf7sP5xtFTr3gtUT4PvCr4cgvte

1 ['EnwbMZFiuCEo2MJ9YEp3PDTqopWZqQ9VLi5x2zASZpX5']
2 ['4ZzDmf2gyKgxh9i8kbULc95p8HQSWsxXJh1waq9o4bhy', '5JGTsLpou5VTcepwwmvEAR2i6CULnMUPEgwGa3e1hrCg']
3 ['26cNzUrVMN9phjfJK8wE9imbuwGKfpkbYLHUUDAxKHLb', '4eWYP6nWEH6WVS6Yw7JD3ENgCmiLCfz5YUEPKCTV1PMH', '9Xea1VRN5Jm1s3yLmFczCFAgVSF7v4fCwF3MGKTYkFz', '3QFodfQQiD5e4vDZ72eh2jFVqtsXPsQXYjxswsaKXECZ']

              _____________Enw..pX5____________
             /                                 \
     ____4Zz..bhy___                     ____5JG..rCg___
    /               \                   /               \
26c..HLb          4eW..PMH          9Xe..kFz          3QF..ECZ



### Check merkle root stored on ICP

In [55]:
reply = nodeApi.rootHash(session, scope, table).get()
data = reply["data"]
display(data)

{'signature': '5SH1PK5NeDJsnLw8ingEkJcb2sX8M1gZSix2hWp1rzMEiTQskbqYnUGbekSN8Dj2jn91VHmj1r5TwLwTVuZ51h1c',
 'rootHash': 'EnwbMZFiuCEo2MJ9YEp3PDTqopWZqQ9VLi5x2zASZpX5',
 'timestamp': '1697632833918'}

### Verify the root hash against ICP

In [56]:
reply = nodeApi.rootHash(session, scope, table).get()
data = reply["data"]
display(data)

chainRootHash = data["rootHash"]
print("\nMatching:", rootHash == chainRootHash) # data hashes are salted. The salt needs to match what's configured on the network to have a match

signingKey = nodeApi.sigKey().get()["data"]
print("Node Public Key:", signingKey)

toSign = rootHash + " " + ts
#print(rootHash)
check = nodeApi.verifySignature(rootHashSignature, toSign)
print("Check signature from merkleTree() call:", check)

toSign = data["rootHash"] + " " + data["timestamp"]
check = nodeApi.verifySignature(data["signature"], toSign)
print("Check signature from blockchain:", check)

# Note, verifySignature isn't unique to Weavechain, it's EdDSA public key cryptography and the verification is done locally

{'signature': '5y2tQAC7msp5LetiLfwutRLkhbueaypyQY7tQYGoDakcAH4htjzcHzUioiTu7XrrEtCSGMrMhZ3tstVMwvku2Xdb',
 'rootHash': 'EnwbMZFiuCEo2MJ9YEp3PDTqopWZqQ9VLi5x2zASZpX5',
 'timestamp': '1697632835303'}


Matching: True
Node Public Key: GfHq2tTVk9z4eXgyHRyb7SpxujN2B86QuKWSEvr91g94fVATX72hejve9YcT
Check signature from merkleTree() call: True
Check signature from blockchain: True


### Verify the presence of a record in the merkle tree

In [57]:
import hashlib
import base64
import hmac

#Compute a hash of the data. Use the same salt that is agreed with the server
row = [ 'Nickname', 'Last Name', 'First name', '1980-01-01', 'email@gmail.com', '+40712345678', 'US', 'Secret', 'EST', 'https://www.linkedin.com/in/linkedin/', 'discord#1234', '@telegram', '0xwallet' ]
recordHash = nodeApi.hashRecord(row, salt, digest)
print("\nData hash:", recordHash)

reply = nodeApi.verifyMerkleHash(session, tree, recordHash, digest).get()
print("Verified to be present in the merkle tree:", reply["data"])


Data hash: 26cNzUrVMN9phjfJK8wE9imbuwGKfpkbYLHUUDAxKHLb
Verified to be present in the merkle tree: true


### Obtain a merkle proof

In [59]:
reply = nodeApi.merkleProof(session, scope, table, recordHash, digest).get()
print(reply)
data = reply["data"]
display(data)

hashes = [ h.split(",") for h in data["proof"].split(";") ]
display(hashes)

proofSignature = data["signature"]
toSign = data["proof"] + " " + data["timestamp"] # the node signed the proof and the timestamp
check = nodeApi.verifySignature(proofSignature, toSign)
print(check)

{'res': 'ok', 'data': {'signature': '4BJduL2S2GXbpDWK2bAPdGwXzUwisW68M2ihuAFnJwtJ9cMz3dBV7WnQjE3QSrudYK1eJiQJRC8iiv7ztq9fkkns', 'rootHash': 'EnwbMZFiuCEo2MJ9YEp3PDTqopWZqQ9VLi5x2zASZpX5', 'proof': '26cNzUrVMN9phjfJK8wE9imbuwGKfpkbYLHUUDAxKHLb,4eWYP6nWEH6WVS6Yw7JD3ENgCmiLCfz5YUEPKCTV1PMH;4ZzDmf2gyKgxh9i8kbULc95p8HQSWsxXJh1waq9o4bhy,5JGTsLpou5VTcepwwmvEAR2i6CULnMUPEgwGa3e1hrCg', 'hash': 'SHA-256', 'timestamp': '1697632842660'}}


{'signature': '4BJduL2S2GXbpDWK2bAPdGwXzUwisW68M2ihuAFnJwtJ9cMz3dBV7WnQjE3QSrudYK1eJiQJRC8iiv7ztq9fkkns',
 'rootHash': 'EnwbMZFiuCEo2MJ9YEp3PDTqopWZqQ9VLi5x2zASZpX5',
 'proof': '26cNzUrVMN9phjfJK8wE9imbuwGKfpkbYLHUUDAxKHLb,4eWYP6nWEH6WVS6Yw7JD3ENgCmiLCfz5YUEPKCTV1PMH;4ZzDmf2gyKgxh9i8kbULc95p8HQSWsxXJh1waq9o4bhy,5JGTsLpou5VTcepwwmvEAR2i6CULnMUPEgwGa3e1hrCg',
 'hash': 'SHA-256',
 'timestamp': '1697632842660'}

[['26cNzUrVMN9phjfJK8wE9imbuwGKfpkbYLHUUDAxKHLb',
  '4eWYP6nWEH6WVS6Yw7JD3ENgCmiLCfz5YUEPKCTV1PMH'],
 ['4ZzDmf2gyKgxh9i8kbULc95p8HQSWsxXJh1waq9o4bhy',
  '5JGTsLpou5VTcepwwmvEAR2i6CULnMUPEgwGa3e1hrCg']]

True
